# Convolutions

In [ ]:
!git clone https://github.com/Srinivas-R/AI4ALL.git

Working example: Identify the presence of squares (of arbitrary size) in a picture

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

np.set_printoptions(threshold=300)

### Load the image

In [ ]:
imagePath = './images2/'
square = np.array(Image.open(imagePath + 'squares2.png'))
plt.imshow(square)

### Plot the R, G and the B channels separately

In [ ]:
plt.title('R Channel')
plt.imshow(square[:,:,0], cmap='gray', vmin=0, vmax=255)
plt.colorbar()

In [ ]:
plt.title('G Channel')
plt.imshow(square[:,:,1], cmap='gray')

In [ ]:
plt.title('B Channel')
plt.imshow(square[:,:,2], cmap='gray')

### For our basic challenge, we don't need the color channels. So let's convert it into grayscale

In [ ]:
gray = 0.2989 * square[:,:,0] + 0.5870 * square[:,:,1] + 0.1140 * square[:,:,2]
plt.imshow(gray, cmap='gray', vmin=0)
plt.colorbar()

In [ ]:
inverted = (1.0 - gray/gray.max())
plt.imshow(inverted, cmap='gray', vmin=0, vmax=1)
plt.colorbar()

### Let's work with a smaller example for now

In [ ]:
small_inverted = inverted[250:450, 0:200]

In [ ]:
plt.imshow(small_inverted, cmap='gray', vmin=0, vmax=1)

### Alright cool, you have everything setup! Now, how do we tackle the actual problem?

### Given size of square: 52 x 52. Recall that the filter resembles what you're searching for: a white square with a black surrounding. Question: Why do we set the surrounding to a large negative value? 

In [ ]:
square_filter = np.ones((53, 53))

In [ ]:
square_filter[:,0] = -15
square_filter[:,-1] = -15
square_filter[0, :] = -15
square_filter[-1, :] = -15

In [ ]:
plt.imshow(square_filter, cmap='gray', vmin=-15, vmax=1)

### Exercise 2.1

Try commenting out the large negative values, see what happens

### Question

Why do I want all this fancy convolution stuff? Why not just search directly (equating the expected value with a sliding window over the image)? 

In [ ]:
inverted_noised = inverted.copy()
inverted_noised[336, 96] = 0.8
for i in range(150):
    x,y = np.random.randint(low=1, high=50), np.random.randint(low=1, high=50)
    inverted_noised[336 + x, 96 + y] = 0.2
small_inverted_noised = inverted_noised[250:450, 0:200]
plt.imshow(small_inverted_noised, cmap='gray')

### Answer

The real world is *noisy*. Smudges from camera, dust and dirt covering the object, effect of lighting, all create *imperfections*. So exact matches are too strict a condition. The square in the image above might have some dirt covering a bit of it, but it should still be counted.

In [ ]:
output = signal.convolve2d(small_inverted, square_filter, mode='valid')
output2 = signal.convolve2d(small_inverted_noised, square_filter, mode='valid')

### Intuition about outputs

Convolutions, when set with the right filter values, gives us a *confidence* value that the filter pattern exists in that location. High value: High confidence. Low value: Low confidence. Then we can set a cut-off value, based on how lenient we want to be. Any location above that threshold: contains the patten 

### Exercise 2.2

Play around with the confidence threshold and see what happens to the imperfect square.

In [ ]:
#increase and decrease this value, run the cell, see what happens
confidence_thresh = 710

vis1 = (output > confidence_thresh)
vis2 = (output2 > confidence_thresh)
prediction = 'Contains square' if vis1.any() else 'No square'
prediction2 = 'Contains square' if vis2.any() else 'No square'

fig = plt.figure(figsize=(15, 15))
ax1 = fig.add_subplot(1, 2, 1)
ax1.title.set_text('perfect square, thresh : {}, Prediction : {}'.format(confidence_thresh, prediction))
plt.imshow(vis1)
ax2 = fig.add_subplot(1, 2, 2)
ax2.title.set_text('imperfect square, thresh : {}, Prediction : {}'.format(confidence_thresh, prediction2))
plt.imshow(vis2)

### Exercise 2.3

Load the triangles image instead of the square and run the above pipeline, see what prediction you get. The steps are given below.

In [ ]:
#Load the triangle image

#Convert to grayscale

#Crop a small portion of the image containing a triangle for convenience

#Convolve using the square filter provided

#See what happens if you use the same confidence value above as a threshold

### Size invariance

So far, we've used exact size of the pattern. But we usually don't know the size. We should try detecting something size-invariant: like corners. So first we convert the filled in square to edges

In [6]:
#create a vertical black to white edge detector
sobel_filter_vertical = np.array([[-1, 0, 1], 
                                 [-2, 0, 2], 
                                 [-1, 0, 1]])

In [7]:
#create a horizontal white to black edge detector
sobel_filter_horizontal = np.array([[1, 2, 1], 
                                    [0, 0, 0], 
                                    [-1,-2,-1]])

In [ ]:
plt.imshow(sobel_filter_vertical, cmap='gray')

In [ ]:
plt.imshow(sobel_filter_horizontal, cmap='gray')

In [ ]:
output = signal.convolve2d(small_inverted, sobel_filter_vertical, mode='valid')
output2 = signal.convolve2d(small_inverted, sobel_filter_horizontal, mode='valid')

In [ ]:
plt.imshow(output, cmap='gray', vmin=-1, vmax=1)

In [ ]:
plt.imshow(output2, cmap='gray', vmin=-1, vmax=1)

### Exercise 2.4

Design filters to detect edges in other direction (white to black).

Hint: Try to emulate the filters above, but in the reverse direction.

In [ ]:
horizontal_filter_inverted = 
vertical_filter_inverted = 

#Convolve the filters over the image, as in the previous cells

output3 = signal.convolve2d(small_inverted, , mode='valid')
output4 = signal.convolve2d(small_inverted, , mode='valid')

#Visualize the outputs as in the previous cells.


In [ ]:
#For each of the convolution outputs, keep a pixel value if positive, make it 0 if negative. Essentially,
#wherever a pattern is found, value is high (and positive), so keep only that. We'll also see some other reasons
#for doing in this in the next lecture.

output_threshed1 = np.maximum(0, )
output_threshed2 = np.maximum(0, )
output_threshed3 = np.maximum(0, )
output_threshed4 = np.maximum(0, )

#Plot each of the above thresholded outputs to see what they look like

#Add all the thresholded outputs to get an edge representation
edge_rep = 

#Plot the new representation (edges of the image).
plt.imshow(edge_rep, cmap='gray', vmin=0, vmax=1)

### It's all about compositions: 

Combine Edges -> Basic Shapes

Combine basic shapes -> Complex Shapes

Combine complex shapes -> Faces/Dogs/Cats/Crop Weeds, etc.

### Examples

Rectange has 4 corner shapes. If we detect these, we can be reasonably confident that a rectange exists.

Triangle has 3 angles. Same as above.

### Exercise 2.5

Design 3 x 3 filters to detect each of the 4 corners of a rectangle. We will be running these on the edge representation, so look at it again to get an idea of what 3 x 3 pixels a corner would have.

In [ ]:
corner_filter1 = np.array()
corner_filter2 = np.array()
corner_filter3 = np.array()
corner_filter4 = np.array()

In [ ]:
#Convolve the filters (over what? fill in below) to detect if corners exist

corner_output1 = signal.convolve2d(, corner_filter1, mode='valid')
corner_output2 = signal.convolve2d(, corner_filter2, mode='valid')
corner_output3 = signal.convolve2d(, corner_filter3, mode='valid')
corner_output4 = signal.convolve2d(, corner_filter4, mode='valid')

In [ ]:
#Set threshold 
confidence_thresh = 

#If pattern exists anywhere in the image, some pixels will exceed confidence thresh
corner1_exists = (corner_output1 > confidence_thresh).any()
corner2_exists = (corner_output2 > confidence_thresh).any()
corner3_exists = (corner_output3 > confidence_thresh).any()
corner4_exists = (corner_output4 > confidence_thresh).any()

#Write the AND function of the above
rectangle_exists = 

In [ ]:
print('Rectangle exists in the following image : ' + str(rectange_exists))